In [25]:
import os
import json
import pandas as pd
import numpy as np
import re
import random
import pickle
import os
from tqdm import tqdm

In [99]:
input_dir = 'original/'
output_dir = './'
states = [ "warm_up", "user_cold_testing", "item_cold_testing", "user_and_item_cold_testing","meta_training"]

ui_data = pd.read_csv(input_dir + 'user_item.dat', names=['user', 'item', 'rating', 'timestamp'],
                      sep='\t', engine='python')
user_time = ui_data[['user','timestamp']]
item_time = ui_data[['item','timestamp']]
user_time = user_time.groupby(['user']).apply(lambda x: x.sort_values(["timestamp"], ascending = True)).reset_index(drop=True).groupby('user').head(1)
item_time = item_time.groupby(['item']).apply(lambda x: x.sort_values(["timestamp"], ascending = True)).reset_index(drop=True).groupby('item').head(1)
user_time.sort_values(by="timestamp" , inplace=True, ascending=False)
item_time.sort_values(by="timestamp" , inplace=True, ascending=False)
user_top = user_time.iloc[:1354]
new_user = user_top['user'].tolist()
item_top = item_time.iloc[:550]
new_item = item_top['item'].tolist()

In [100]:
user_list = set(ui_data.user)
item_list = set(ui_data.item)
len(user_list), len(item_list)

(6170, 2753)

In [101]:
exist_user = list(user_list - set(new_user))
exist_item = list(item_list- set(new_item))
len(exist_user), len(exist_item)

(4816, 2203)

meta_training and meta-testing data

In [102]:
meta_training_data = ui_data[(ui_data['user'].isin(exist_user)) & (ui_data['item'].isin(exist_item))]
len(meta_training_data)  # 967291 

135258

In [103]:
warm_data = meta_training_data.sample(int(0.1*len(meta_training_data)))
len(warm_data)  # 96729 

13525

In [104]:
count_user_interaction = dict(warm_data.user.value_counts())
less_100_user = [k for k, v in count_user_interaction.items() if 13<=v<=100]
len(less_100_user)

154

In [105]:
warm_data = warm_data[warm_data['user'].isin(less_100_user)]#124599

In [106]:
# warm data
warm_x = {k: g["item"].tolist() for k,g in warm_data.groupby("user")}
warm_y = {k: g["rating"].tolist() for k,g in warm_data.groupby("user")}
json.dump(warm_x, open("{}/warm_up.json".format(output_dir), 'w'))
json.dump(warm_y, open("{}/warm_up_y.json".format(output_dir), 'w'))
len(warm_x), len(warm_y)

(154, 154)

In [107]:
training_data = meta_training_data.loc[meta_training_data.index.difference(warm_data.index)]
len(training_data)  # 870562 

132045

In [108]:
count_user_interaction = dict(training_data.user.value_counts())
less_100_user = [k for k, v in count_user_interaction.items() if 13<=v<=100]
training_data = training_data[training_data['user'].isin(less_100_user)]#124599
len(less_100_user)

3012

In [109]:
# training data
training_x = {k: g["item"].tolist() for k,g in training_data.groupby("user")}
training_y = {k: g["rating"].tolist() for k,g in training_data.groupby("user")}
json.dump(training_x, open("{}/meta_training.json".format(output_dir), 'w'))
json.dump(training_y, open("{}/meta_training_y.json".format(output_dir), 'w'))
len(training_x), len(training_y)

(3012, 3012)

In [110]:
# meta-testing
# user_cold_testing
user_cold_data =ui_data[(ui_data['user'].isin(new_user)) & (ui_data['item'].isin(exist_item))]
len(user_cold_data) 

25885

In [111]:
count_user_interaction = dict(user_cold_data.user.value_counts())
less_100_user = [k for k, v in count_user_interaction.items() if 13<=v<=100]
user_cold_data = user_cold_data[user_cold_data['user'].isin(less_100_user)]#124599
len(less_100_user)

719

In [112]:
user_cold_x = {k: g["item"].tolist() for k,g in user_cold_data.groupby("user")}
user_cold_y = {k: g["rating"].tolist() for k,g in user_cold_data.groupby("user")}
json.dump(user_cold_x, open("{}/user_cold_testing.json".format(output_dir), 'w'))
json.dump(user_cold_y, open("{}/user_cold_testing_y.json".format(output_dir), 'w'))
len(user_cold_x), len(user_cold_y)

(719, 719)

In [113]:
# item_cold_testing
item_cold_data =ui_data[(ui_data['user'].isin(exist_user)) & (ui_data['item'].isin(new_item))]
len(item_cold_data)  # 118467 

30712

In [114]:
count_user_interaction = dict(item_cold_data.user.value_counts())
less_100_user = [k for k, v in count_user_interaction.items() if 13<=v<=100]
item_cold_data = item_cold_data[item_cold_data['user'].isin(less_100_user)]#124599
len(less_100_user)

639

In [115]:
item_cold_x = {k: g["item"].tolist() for k,g in item_cold_data.groupby("user")}
item_cold_y = {k: g["rating"].tolist() for k,g in item_cold_data.groupby("user")}
json.dump(item_cold_x, open("{}/item_cold_testing.json".format(output_dir), 'w'))
json.dump(item_cold_y, open("{}/item_cold_testing_y.json".format(output_dir), 'w'))
len(item_cold_x), len(item_cold_y)

(639, 639)

In [123]:
user_item_cold_data =ui_data[(ui_data['user'].isin(new_user)) & (ui_data['item'].isin(new_item))]
len(user_item_cold_data)  # 52736 

3936

In [124]:
count_user_interaction = dict(user_item_cold_data.user.value_counts())
less_10_user = [k for k, v in count_user_interaction.items() if 13<=v<=100]
user_item_cold_data = user_item_cold_data[user_item_cold_data['user'].isin(less_10_user)]#124599
len(less_10_user)

25

In [125]:
user_item_cold_x = {k: g["item"].tolist() for k,g in user_item_cold_data.groupby("user")}
user_item_cold_y = {k: g["rating"].tolist() for k,g in user_item_cold_data.groupby("user")}
json.dump(user_item_cold_x, open("{}/user_and_item_cold_testing.json".format(output_dir), 'w'))
json.dump(user_item_cold_y, open("{}/user_and_item_cold_testing_y.json".format(output_dir), 'w'))
len(user_item_cold_x), len(user_item_cold_y)

(25, 25)

In [79]:
ui_data_filter = ui_data[ui_data['user_id'].isin(less_100_user)]#124599

In [82]:
count_user_interaction = dict(ui_data_filter.user_id.value_counts())


In [83]:
array = ui_data_filter["user_id"].unique()
d = dict(enumerate(array))
res = dict((v, k) for k, v in d.items())
ui_data_filter['user'] = ui_data_filter["user_id"].map(res)

C:\Users\administered\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [85]:
ui_data_filter

,user_id,item,rating,timestamp,user
0,2836,2645,5,1386201600,0
3,3044,2645,5,1012176000,1
4,2861,2645,4,1388966400,2
6,4438,2645,5,1391817600,3
7,4276,2645,4,1003449600,4
9,2606,2645,4,1388793600,5
12,4449,2645,4,1354233600,6
14,4379,2645,5,1387065600,7
15,388,2616,5,1077321600,8
16,3680,2616,5,1232582400,9


In [84]:
count_user_interaction = dict(ui_data_filter.user.value_counts())
less_100_user = [k for k, v in count_user_interaction.items() if 13<=v<=100]
len(less_100_user)

4579

In [86]:
user_list = set(ui_data_filter.user)
item_list = set(ui_data_filter.item)
len(user_list), len(item_list)

(4579, 2753)

In [87]:
user_time = ui_data_filter[['user','timestamp']]
item_time = ui_data_filter[['item','timestamp']]
user_time = user_time.groupby(['user']).apply(lambda x: x.sort_values(["timestamp"], ascending = True)).reset_index(drop=True).groupby('user').head(1)
item_time = item_time.groupby(['item']).apply(lambda x: x.sort_values(["timestamp"], ascending = True)).reset_index(drop=True).groupby('item').head(1)
user_time.sort_values(by="timestamp" , inplace=True, ascending=False)
item_time.sort_values(by="timestamp" , inplace=True, ascending=False)

In [88]:
user_top = user_time.iloc[:915]
new_user = user_top['user'].tolist()
item_top = item_time.iloc[:550]
new_item = item_top['item'].tolist()

In [90]:
exist_user = list(user_list - set(new_user))
exist_item = list(item_list- set(new_item))
len(exist_user), len(exist_item)

(3664, 2203)

meta_training and meta-testing data

In [91]:
meta_training_data = ui_data_filter[(ui_data_filter['user'].isin(exist_user)) & (ui_data_filter['item'].isin(exist_item))]
len(meta_training_data)  # 967291 

84924

In [57]:
warm_data = meta_training_data.sample(int(0.1*len(meta_training_data)))
len(warm_data)  # 96729 

8492

In [58]:
# warm data
warm_x = {k: g["item"].tolist() for k,g in warm_data.groupby("user")}
warm_y = {k: g["rating"].tolist() for k,g in warm_data.groupby("user")}
json.dump(warm_x, open("{}/warm_up.json".format(output_dir), 'w'))
json.dump(warm_y, open("{}/warm_up_y.json".format(output_dir), 'w'))
len(warm_x), len(warm_y)

(3012, 3012)

In [60]:
training_data = meta_training_data.loc[meta_training_data.index.difference(warm_data.index)]
len(training_data)  # 870562 

76432

In [61]:
# training data
training_x = {k: g["item"].tolist() for k,g in training_data.groupby("user")}
training_y = {k: g["rating"].tolist() for k,g in training_data.groupby("user")}
json.dump(training_x, open("{}/meta_training.json".format(output_dir), 'w'))
json.dump(training_y, open("{}/meta_training_y.json".format(output_dir), 'w'))
len(training_x), len(training_y)

(3663, 3663)

In [11]:
count_user_interaction = dict(training_data.user.value_counts())
less_100_user = [k for k, v in count_user_interaction.items() if 13<=v<=100]
len(less_100_user)

2662

In [63]:
# meta-testing
# user_cold_testing
user_cold_data =ui_data_filter[(ui_data_filter['user'].isin(new_user)) & (ui_data_filter['item'].isin(exist_item))]
len(user_cold_data) 

18240

In [64]:
user_cold_x = {k: g["item"].tolist() for k,g in user_cold_data.groupby("user")}
user_cold_y = {k: g["rating"].tolist() for k,g in user_cold_data.groupby("user")}
json.dump(user_cold_x, open("{}/user_cold_testing.json".format(output_dir), 'w'))
json.dump(user_cold_y, open("{}/user_cold_testing_y.json".format(output_dir), 'w'))
len(user_cold_x), len(user_cold_y)

(915, 915)

In [65]:
count_user_interaction = dict(user_cold_data.user.value_counts())
less_100_user = [k for k, v in count_user_interaction.items() if 13<=v<=100]
len(less_100_user)

698

In [66]:
# item_cold_testing
item_cold_data =ui_data_filter[(ui_data_filter['user'].isin(exist_user)) & (ui_data_filter['item'].isin(new_item))]
len(item_cold_data)  # 118467 

18694

In [67]:
item_cold_x = {k: g["item"].tolist() for k,g in item_cold_data.groupby("user")}
item_cold_y = {k: g["rating"].tolist() for k,g in item_cold_data.groupby("user")}
json.dump(item_cold_x, open("{}/item_cold_testing.json".format(output_dir), 'w'))
json.dump(item_cold_y, open("{}/item_cold_testing_y.json".format(output_dir), 'w'))
len(item_cold_x), len(item_cold_y)

(2546, 2546)

In [68]:
count_user_interaction = dict(item_cold_data.user.value_counts())
less_100_user = [k for k, v in count_user_interaction.items() if 13<=v<=100]
len(less_100_user)

428

In [69]:
# user_and_item_cold_testing
user_item_cold_data =ui_data_filter[(ui_data_filter['user'].isin(new_user)) & (ui_data_filter['item'].isin(new_item))]
len(user_item_cold_data)  # 52736 

2741

In [70]:
user_item_cold_x = {k: g["item"].tolist() for k,g in user_item_cold_data.groupby("user")}
user_item_cold_y = {k: g["rating"].tolist() for k,g in user_item_cold_data.groupby("user")}
json.dump(user_item_cold_x, open("{}/user_and_item_cold_testing.json".format(output_dir), 'w'))
json.dump(user_item_cold_y, open("{}/user_and_item_cold_testing_y.json".format(output_dir), 'w'))
len(user_item_cold_x), len(user_item_cold_y)

(771, 771)

In [71]:
count_user_interaction = dict(user_item_cold_data.user.value_counts())
less_10_user = [k for k, v in count_user_interaction.items() if 13<=v<=100]
len(less_10_user)

16

In [21]:
len(training_data)+len(warm_data)+len(user_cold_data)+len(item_cold_data)+len(user_item_cold_data)

195791